In [1]:
import os
import torch
import imageio.v3 as iio
import numpy as np

from base64 import b64encode
from cotracker.utils.visualizer import Visualizer, read_video_from_path
from IPython.display import HTML
from cotracker.predictor import CoTrackerPredictor, CoTrackerOnlinePredictor
import PoseEstimation.customutils as customutils

# model = torch.hub.load("facebookresearch/co-tracker", "cotracker2_online").to("cuda")
DEFAULT_DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CoTrackerPredictor(checkpoint=os.path.join('./co-tracker/checkpoints/cotracker2.pth'))#.to(DEFAULT_DEVICE)

NUM_KEYPOINTS = 23#26

if torch.cuda.is_available():
    model = model.cuda()

df_running_annotations = customutils.load_images_dataframe()
df_running_annotations

,image_id,pose_id,file_name,frame,person
0,1000,1000,Athletics_Mixed_Tokyo_2020_20_1.mp4,0,0
1,1001,1001,Athletics_Mixed_Tokyo_2020_20_1.mp4,1,0
2,1002,1002,Athletics_Mixed_Tokyo_2020_20_1.mp4,2,0
3,1003,1003,Athletics_Mixed_Tokyo_2020_20_1.mp4,3,0
4,1004,1004,Athletics_Mixed_Tokyo_2020_20_1.mp4,4,0
...,...,...,...,...,...
12105,45160,145160,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,160,1
12106,45161,145161,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,161,1
12107,45162,145162,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,162,1
12108,45163,145163,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,163,1


In [2]:
def _process_step(window_frames, is_first_step, queries = None):
        video_chunk = (
            torch.tensor(np.stack(window_frames[-model.step * 2 :]), device=DEFAULT_DEVICE)
            .float()
            .permute(0, 3, 1, 2)[None]
        )  # (1, T, 3, H, W)
        if queries is None or len(queries) == 0:
            return model(
            video_chunk,
            is_first_step=is_first_step
            )
        else:
            if torch.cuda.is_available():
                queries = queries.cuda()
                
            return model(
                video_chunk,
                is_first_step=is_first_step,
                queries=queries[None]
            )

In [3]:
import cv2

def crop_video(input_path, output_path, x_start, y_start, width, height):
    # Open the video file
    cap = cv2.VideoCapture(input_path)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        print("Error: Could not open video file.")
        exit()

    # Get the original video dimensions
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create VideoWriter object to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec based on your requirements
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (width, height))

    # Iterate through the frames
    while True:
        ret, frame = cap.read()

        # Break the loop if we reach the end of the video
        if not ret:
            break

        # Crop the frame to the specified dimensions
        cropped_frame = frame[y_start:y_start + height, x_start:x_start + width]

        # Resize the cropped frame to the new dimensions
        resized_frame = cv2.resize(cropped_frame, (width, height))

        # Write the resized frame to the output video file
        out.write(resized_frame)

    # Release video capture and writer objects
    cap.release()
    out.release()

    # Close any open windows
    cv2.destroyAllWindows()


In [4]:
import cv2
import csv
import time
import peakutils
from KeyFrameDetector.utils import convert_frame_to_grayscale

def get_keyframes(filename):
    cap = cv2.VideoCapture('./data/videos/' + filename)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    lstfrm = []
    lstdiffMag = []
    timeSpans = []
    images = []
    full_color = []
    lastFrame = None

    for i in range(length):
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        grayframe, blur_gray = convert_frame_to_grayscale(frame)

        frame_number = cap.get(cv2.CAP_PROP_POS_FRAMES) - 1
        lstfrm.append(frame_number)
        images.append(grayframe)
        full_color.append(frame)
        if frame_number == 0:
            lastFrame = blur_gray

        diff = cv2.subtract(blur_gray, lastFrame)
        diffMag = cv2.countNonZero(diff)
        lstdiffMag.append(diffMag)
        lastFrame = blur_gray

    
    y = np.array(lstdiffMag)
    base = peakutils.baseline(y, 2)
    indices = peakutils.indexes(y-base, float(0.3), min_dist=1)

    # reset to first frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    cap.release()
    cv2.destroyAllWindows()
    
    return indices

def cotracker_model(video, filename, person_count, queries_dict, indices, id, visualize = True):
    # Iterating over video frames, processing one window at a time:
    is_first_step = True

    queries = queries_dict[0]
    idx = []
    #indices = [0]
    for i, ind in enumerate(indices):
        if ind == 0:
            continue
        if queries_dict[ind] is None or len(queries_dict[ind]) == 0:
            idx.append(i)
        #elif ind < len(queries_dict):
        else:
            queries = torch.vstack((queries, queries_dict[ind]))
    indices = np.delete(indices, idx, axis=0)

    if torch.cuda.is_available():
        queries = queries.cuda()
        
    pred_tracks, pred_visibility = model(
                video,
                queries=queries[None]
            )
    
    new_pred_tracks = None    
    new_pred_visibility = None    
    keyframe_i = 0
    total_kpts_count=NUM_KEYPOINTS*person_count

    #indices = np.insert(indices, 0, 0)
    for i in range(len(pred_tracks[0])):
        frame = pred_tracks[0][i]
        frame_vis = pred_visibility[0][i]

        j = keyframe_i*total_kpts_count
        k = (keyframe_i+1)*total_kpts_count
        if k > frame.shape[0]:
            k = frame.shape[0]-1
        new_frame = frame[j:k]
        new_frame_vis = frame_vis[j:k]
        
        if new_frame.shape[0] < total_kpts_count:
            new_pred_tracks = torch.cat((new_pred_tracks, frame.unsqueeze(0)),dim=0)
            new_pred_visibility = torch.cat((new_pred_visibility, frame_vis.unsqueeze(0)),dim=0)
            continue

        if new_pred_tracks is None:
            new_pred_tracks = new_frame
            new_pred_visibility = new_frame_vis
        elif len(new_pred_tracks.shape) == 2:
            new_pred_tracks = torch.stack((new_pred_tracks, new_frame),dim=0)
            new_pred_visibility = torch.stack((new_pred_visibility, new_frame_vis),dim=0)
        else:
            new_pred_tracks = torch.cat((new_pred_tracks, new_frame.unsqueeze(0)),dim=0)
            new_pred_visibility = torch.cat((new_pred_visibility, new_frame_vis.unsqueeze(0)),dim=0)
            
        if keyframe_i+1 < len(indices) and indices[keyframe_i+1] <= i+1:
            keyframe_i += 1
            
    if visualize:
        vis = Visualizer(save_dir="./videos/results/cotracker3/", linewidth=3)
        vis.visualize(video, torch.unsqueeze(new_pred_tracks, 0), torch.unsqueeze(new_pred_visibility, 0), filename=filename.replace('.mp4', '') + '_' + str(id), query_frame=0)
    
    return torch.unsqueeze(new_pred_tracks, 0), torch.unsqueeze(new_pred_visibility, 0)

In [5]:
import PoseEstimation.customutils as customutils

df_running_annotations = customutils.load_images_dataframe()
filenames = df_running_annotations['file_name'].unique()
filenames

array(['Athletics_Mixed_Tokyo_2020_20_1.mp4',
       'Athletics_Mixed_Tokyo_2020_39.mp4',
       'Athletics_Mixed_Tokyo_2020_5.mp4',
       'Athletics_Mixed_Tokyo_2020_64.mp4',
       'Athletics_Mixed_Tokyo_2020_8.mp4', 'clip_tri_1.mp4',
       'Marathon_Men_Tokyo_2020_14.mp4', 'Marathon_Men_Tokyo_2020_16.mp4',
       'Marathon_Men_Tokyo_2020_28.mp4', 'Marathon_Men_Tokyo_2020_47.mp4',
       'Marathon_Men_Tokyo_2020_52.mp4',
       'Triathlon_Men_Tokyo_2020_12_1.mp4',
       'Triathlon_Men_Tokyo_2020_19_1.mp4',
       'Triathlon_Men_Tokyo_2020_21_1.mp4',
       'Triathlon_Men_Tokyo_2020_23.mp4',
       'Triathlon_Men_Tokyo_2020_28.mp4',
       'Triathlon_Women_Tokyo_2020_29.mp4',
       'Triathlon_Women_Tokyo_2020_2_1.mp4',
       'Triathlon_Women_Tokyo_2020_33_1.mp4',
       'World_Athletics_Men_10000m_Oregon_2022_2.mp4',
       'World_Athletics_Men_10000m_Oregon_2022_23.mp4',
       'World_Athletics_Men_10000m_Oregon_2022_25.mp4',
       'World_Athletics_Men_10000m_Oregon_2022_28.mp4

Import CoTrackerPredictor and create an instance of it. We'll use this object to estimate tracks:

In [6]:

def load_video(filename):
    video = read_video_from_path('./data/videos/' + filename)
    video = torch.from_numpy(video).permute(0, 3, 1, 2)[None].float()

    if torch.cuda.is_available():
        video = video.cuda()
    
    return video

Tracking manually selected points

In [7]:
import os, argparse, json, re
from collections import defaultdict
import pandas as pd

def load_annotations(file_path):
    frames = defaultdict(list)
    isExist = os.path.exists(file_path)
    person_count = 0
    if isExist:
      annotations = []
      with open(file_path, 'r') as f:
          annotations = json.load(f)
          
          keypoints = defaultdict(list)
          person_count = len(annotations['annotations'])
          for person in annotations['annotations']:
            framecount = len(person['frames'])
            for frame_index in range(0, framecount):
              points = []
              if frame_index < framecount:
                frame = person['frames'][str(frame_index)]
                for node in frame['skeleton']['nodes']:
                    points.append({'id': node['name'], 'x' : node['x'], 'y': node['y']})

              if len(points) > 0:
                keypoints[frame_index].append({'person': {'points': points}})

          for frame in range(0, len(keypoints)):
            frames[frame] = keypoints[frame]

    return frames, person_count


def get_queries_for_frame(frame_number, annotations):
  i = 0

  for person in annotations[frame_number]:
    for point in person['person']['points']:
      new_tensor = torch.tensor([float(frame_number), point['x'], point['y']])
      if i == 0:
        queries_for_frames = new_tensor
      else:
        queries_for_frames = torch.vstack((queries_for_frames, new_tensor))
      i += 1

  return queries_for_frames


def get_queries_for_frames(start_frame, end_frame, annotations, step = 4):
  frame = 0
  frames_dict = defaultdict(list)
  all_queries_for_frames = None

  if end_frame == -1:
    end_frame = len(annotations)

  for frame in range(start_frame, end_frame, step):
    queries_for_frames = None
    for person in annotations[frame]:
      for point in person['person']['points']:
        if 'occluded' in point and point['occluded'] == 'false':
          continue
        new_tensor = torch.tensor([float(frame), point['x'], point['y']])
        if queries_for_frames is None:
          queries_for_frames = new_tensor
        if all_queries_for_frames is None:
          queries_for_frames = new_tensor
          all_queries_for_frames = new_tensor
        else:
          queries_for_frames = torch.vstack((queries_for_frames, new_tensor))
          all_queries_for_frames = torch.vstack((all_queries_for_frames, new_tensor))

    frames_dict[frame] = queries_for_frames

  return all_queries_for_frames, frames_dict

def get_bboxs(df, df_annotations):
  if df_annotations is not None and len(df_annotations) > 0:
    # Find closest matching pose for first frame     
    #gts = df_annotations[df_annotations['image_id'] == df['image_id']]
    id_box = defaultdict(list)
    idxs = []
    first_frame = df_annotations[df_annotations['image_id'] == df_annotations['image_id'].iloc[0]]

    for pose_id in first_frame.index:
      poses = df[df['image_id'] == df_annotations['image_id'][pose_id]]
      min_dist = np.inf
      gt = df_annotations['keypoints'][pose_id]
      kpts2, vi2 = customutils.edit_keypoints(gt)

      # compute head size for distance normalization
      head = customutils.get_keypoint(gt,"head")
      neck = customutils.get_keypoint(gt,"neck")

      headSize = 1
      if (len(head) > 0 and len(neck) > 0):
          headSize = customutils.get_head_size(head[0], head[1], neck[0], neck[1])
          
      for pose in poses.iloc:
        # Threshold to consider valid pose
        if pose['score'] > .8:
          dt = pose['keypoints']
          kpts1, vi1 = customutils.edit_keypoints(dt)
          d = np.linalg.norm(kpts1 - kpts2, ord=2, axis=1)
          v = np.ones(len(d))

          for part in range(len(d)):
              if vi1[part] == 0 or vi2[part] == 0:
                  d[part] = 0
                  v[part] = 0

          # normalize distance
          dNorm = np.sum(d)/headSize

          if dNorm < min_dist:
            min_dist = dNorm
            idx = pose['idx']

      idxs.append(idx)
    
    # For each pose in tracking
    for pose in df.iloc:
      if pose['idx'] not in idxs:
        continue

      frame = pose['image_id'] % 1000
      idx = pose['idx']
      bbox = np.array(customutils.get_bbox_coord(pose['box']))
      if id_box[idx] is None or len(id_box[idx]) == 0:
        id_box[idx] = defaultdict(list)
        id_box[idx][frame] = bbox
      else:
        id_box[idx][frame] = bbox

  return idxs, id_box

def get_queries_from_pe(start_frame, end_frame, indices, df, boxes):
  frames_dict = defaultdict(list)

  if end_frame == -1:
    end_frame = len(df)
  
  #for frame in range(start_frame, end_frame):
  for frame in indices:
    bbox = boxes[frame]
    if bbox is None or len(bbox) == 0:
      continue

    queries_for_frames = None
    poses = df[df['image_id'] % 1000 == frame]
    min_dist = np.inf
    selected_pose = None
    for pose in poses.iloc:
      box = np.array(pose['box'][:-1])
      d = np.linalg.norm(bbox - box, ord=2)
      d = np.sum(d)
      if d < min_dist:
        min_dist = d
        selected_pose = pose

    if selected_pose is not None:
      kpts = selected_pose['keypoints']
      x, y, vi = customutils.get_x_y_v_keypoints(kpts)
      for point in range(len(x)):
        #if vi[point] == 0:
        if x[point] == float(0) or y[point] == float(0): #skip head, neck, hip
          continue
        new_tensor = torch.tensor([float(frame), float(x[point]), float(y[point])])
        if queries_for_frames is None:
          queries_for_frames = new_tensor
        else:
          queries_for_frames = torch.vstack((queries_for_frames, new_tensor))

    frames_dict[frame] = queries_for_frames

  return frames_dict

def get_idxs(df, df_annotations):
  if df_annotations is not None and len(df_annotations) > 0:
    # Find closest matching pose for first frame 
    poses = df[df['frame'] == 0]
    gts = df_annotations[df_annotations['image_id'] == poses['image_id'].iloc[0]]
    idxs = []
    
    # For each pose in ground truth
    for pose_id in gts.index:
        min_dist = np.inf
        gt = gts['keypoints'][pose_id]
        kpts2, vi2 = customutils.edit_keypoints(gt)
                                
        area = customutils.compute_area_keypoints(gt)

        # compute head size for distance normalization
        head = customutils.get_keypoint(gt,"head")
        neck = customutils.get_keypoint(gt,"neck")

        headSize = 1
        if (len(head) > 0 and len(neck) > 0):
            headSize = customutils.get_head_size(head[0], head[1], neck[0], neck[1])
            
        for pose in poses.iloc:
          # Threshold to consider valid pose
          if pose['score'] > .8:
            dt = pose['keypoints']
            kpts1, vi1 = customutils.edit_keypoints(dt)
            d = np.linalg.norm(kpts1 - kpts2, ord=2, axis=1)
            v = np.ones(len(d))

            for part in range(len(d)):
                if vi1[part] == 0 or vi2[part] == 0:
                    d[part] = 0
                    v[part] = 0

            # normalize distance
            dNorm = np.sum(d)/headSize

            if dNorm < min_dist:
              min_dist = dNorm
              idx = pose['idx']

        # Add closest matching pose
        idxs.append(idx)

  return idxs

def get_queries_from_pe_idx(start_frame, end_frame, df, idx):
  frames_dict = defaultdict(list)

  if end_frame == -1:
    end_frame = len(df)
  
  for frame in range(start_frame, end_frame):
    queries_for_frames = None
    poses = df[df['frame'] == frame]
    for pose in poses.iloc:

      if idx == -1:
        idx = pose['idx']

      if idx != pose['idx']:
        continue

      kpts = pose['keypoints']
      x, y, vi = customutils.get_x_y_v_keypoints(kpts)
      for point in range(len(x)):
        if vi[point] == 0:
          continue
        new_tensor = torch.tensor([float(frame), float(x[point]), float(y[point])])
        if queries_for_frames is None:
          queries_for_frames = new_tensor
        else:
          queries_for_frames = torch.vstack((queries_for_frames, new_tensor))

      break

    frames_dict[frame] = queries_for_frames

  return frames_dict

In [8]:
df_annotations = customutils.load_keypoints_dataframe()
df_annotations

,id,image_id,category_id,bbox,area,iscrowd,num_keypoints,keypoints
0,1000,1000,1,"[957.5031, 204.6478, 294.4987, 641.9175]",189043.869257,1,26,"[1132.7201, 271.7583, 1, 1151.8834, 256.5806, ..."
1,1001,1001,1,"[895.2875, 201.4572, 396.59609999999986, 639.6...",253699.828317,1,26,"[1135.4403, 262.4473, 1, 1151.8834, 253.39, 1,..."
2,1002,1002,1,"[876.5417, 207.8383, 455.8919000000001, 641.2885]",292358.232713,1,26,"[1132.7201, 255.8056, 1, 1148.6928, 247.0089, ..."
3,1003,1003,1,"[892.6316, 187.0998, 445.91919999999993, 658.8...",293787.800419,1,26,"[1129.5295, 249.4245, 1, 1145.5023, 240.6278, ..."
4,1004,1004,1,"[912.8355, 201.4572, 406.572, 681.1703]",276944.771212,1,26,"[1132.7201, 249.4245, 1, 1148.6928, 240.6278, ..."
...,...,...,...,...,...,...,...,...
12105,145160,45160,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,1,26,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,..."
12106,145161,45161,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,1,26,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,..."
12107,145162,45162,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,1,26,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,..."
12108,145163,45163,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,1,26,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,..."


In [9]:
def get_image_id(filename):
    mask_file = df_running_annotations['file_name'] == filename
    mask_frame = df_running_annotations['frame'] == 0
    image_id = df_running_annotations.loc[mask_file & mask_frame]['image_id'].iloc[0]
    return image_id
    
def run_cotracker(filename, video, df_det, df_pe):
    frames = []
    annot_json = 'videos/annotations/' + filename.replace('.mp4', '') + '.json'
    annotations, person_count = load_annotations(annot_json)
    df_video_imageids = df_running_annotations[df_running_annotations['file_name'] == filename]
    
    df_gt = df_annotations[df_annotations['image_id'].isin(df_video_imageids['image_id'])]
    df = df_det[df_det['image_id'].isin(df_video_imageids['image_id'])]
    idxs, idx_boxes = get_bboxs(df, df_gt)

    indices = get_keyframes(filename)
    indices = np.insert(indices, 0, 0)
    df = df_pe[df_pe['image_id'].isin(df_video_imageids['image_id'])]

    for idx in idxs:
        queries_dict = get_queries_from_pe(0, -1, indices, df, idx_boxes[idx])
        if len(queries_dict) == 0:
            print('No queries found for ' + str(idx))
            continue
        print('Processing ' + str(idx))
        pred_tracks, pred_visibility = cotracker_model(video, filename, 1, queries_dict, indices, idx)

        num_frames = pred_tracks.cpu().shape[1]
        np_pred = pred_tracks.cpu().numpy()
        np_vis = pred_visibility.cpu().numpy()
        #df = pd.DataFrame(index=range(num_frames), columns=[str(x) for x in range(num_frames)])
        for i in range(num_frames):
            frame = {}
            frame['image_id'] = int(get_image_id(filename)) + i
            frame['idx'] = int(idx)
            frame['category_id'] = 1

            keypoints = []
            for j in range(len(np_pred[0][i])):
                if j % NUM_KEYPOINTS == NUM_KEYPOINTS:
                    # save current set of keypoints and start new frame/pose
                    frame['keypoints'] = keypoints
                    frames.append(frame)
                    keypoints = []
                keypoints.append(np_pred[0][i][j][0].astype(float))
                keypoints.append(np_pred[0][i][j][1].astype(float))
                if np_vis[0][i][j]:
                    keypoints.append(2)
                else:
                    keypoints.append(1)

            frame['keypoints'] = keypoints
            frames.append(frame)

    customutils.writeJson({'frames': frames, 'keyframes': indices.tolist()},'videos/results/cotracker3/' + filename.replace('.mp4', '') + '.json')

    return frames

In [10]:

df_pe_alphapose = customutils.load_pe_dataframe('alphapose')
df_pe_openpose = customutils.load_pe_dataframe('openpose')
df_pe_ViTPose = customutils.load_pe_dataframe('ViTPose')

df_pe_alphapose

,image_id,keypoints,box,idx,score,category_id
0,1000,"[672.5870361328125, 397.4309997558594, 2, 692....","[455.3162841796875, 340.884765625, 402.0437011...",1,0.898572,1
1,1000,"[1136.07421875, 266.266845703125, 2, 1155.6085...","[932.1016845703125, 190.0831756591797, 310.273...",2,0.877560,1
2,1001,"[682.8905029296875, 407.08642578125, 2, 702.61...","[456.44172066815923, 337.55727539062497, 405.4...",1,0.891666,1
3,1001,"[1134.0335693359375, 257.7659912109375, 2, 115...","[938.4007471361167, 194.7648895263672, 306.255...",2,0.867543,1
4,1002,"[691.739990234375, 412.57452392578125, 2, 713....","[443.5269343344591, 336.3762005969505, 431.860...",1,0.859866,1
...,...,...,...,...,...,...
45470,45163,"[704.8864135742188, 428.4450988769531, 2, 708....","[604.3679742712545, 384.44794226840435, 129.43...",29,0.775203,1
45471,45163,"[242.5023956298828, 475.6257019042969, 2, 245....","[219.46201027790286, 441.22925448208105, 108.8...",12,0.555517,1
45472,45163,"[1689.3052978515625, 480.3915100097656, 2, 168...","[1669.4820232953628, 467.83948263581397, 20.96...",37,0.275808,1
45473,45163,"[1490.57373046875, 472.3777160644531, 2, 1493....","[1481.6382385788675, 450.34171338820823, 32.94...",54,0.676191,1


In [12]:
frames = []

for filename in filenames:
    if filename == "Marathon_Men_Tokyo_2020_14.mp4" or filename == "Triathlon_Men_Tokyo_2020_19_1.mp4":
        continue
    # if filename == "Triathlon_Men_Tokyo_2020_28.mp4":
        # continue
    # if filename != "Athletics_Mixed_Tokyo_2020_8.mp4":
        # continue
    isExist = os.path.exists("./videos/results/cotracker3/" + filename.replace('.mp4','.json'))
    if not isExist:    
        print("Processing " + filename)
        video = load_video(filename)
        frames += run_cotracker(filename, video, df_pe_alphapose, df_pe_ViTPose)

    
customutils.writeJson(frames,'videos/results/cotracker3/person_keypoints_running.json')

Processing Triathlon_Men_Tokyo_2020_28.mp4
